# Interagindo com Dados Externos

Nesta seção, veremos como usar o Python para interagir com dados externos de diversas fontes. 

## Arquivos em Disco

### Abrindo Arquivos para Leitura

Em anexo, na pasta `./Dados`, nós temos uma cópia do livro [Dom Casmurro][dom_casmurro] de [Machado de Assis][machado_de_assis] obtido no site do [Projeto Gutenberg][gutenberg]. Para abri-lo, usamos o comando `open` do Python.


[dom_casmurro]: https://pt.wikipedia.org/wiki/Dom_Casmurro
[machado_de_assis]: https://pt.wikipedia.org/wiki/Machado_de_Assis
[gutenberg]: http://www.gutenberg.org/wiki/Main_Page

In [ ]:
dom_casmurro = open('Dados/machado_de_assis_dom_casmurro.txt')
dom_casmurro

Para ler todo o conteúdo do arquivo, basta usar o método `read`.

In [ ]:
dom_casmurro.read()[:1000] # Mostrando só os primeiros 1000 caracteres

Ao invés de retornar todo o arquivo como uma `string`, podemos usar o método `readlines` para nos retornar cada linha em uma `string` dentro de uma lista.

In [ ]:
dom_casmurro.seek(0) # retorna o início do arquivo
dom_casmurro.readlines()[:10] # só mostra as 10 primeiras linhas

Se o arquivo for muito grande, pode ser interessante ler linha por linha e para isso usamos o método `readline`

In [ ]:
dom_casmurro.seek(0)

for i in range(10):
    print(dom_casmurro.readline())

Ao terminar com o arquivo, devemos sempre fechá-lo para liberar o espaçõ em memória:

In [ ]:
dom_casmurro.close()

E podemos verificar se o arquivo foi fechado com sucesso:

In [ ]:
dom_casmurro.closed

### Escrevendo em Arquivos

O método `open` para abrir arquivos possui outros modos. Acima, utilizamos o modo de leitura `r` que é o padrão. Vamos usar agora o modo `w` de escrita.

In [ ]:
meu_texto = open('Dados/meu_texto.txt', 'w')

In [ ]:
meu_texto.write('1\n')
meu_texto.writelines(['2\n', '3\n']) # Usado para escrever uma lista de strings

In [ ]:
meu_texto.close()

Ao usar o modo `w`, o conteúdo do arquivo é limpo ao ser aberto. Se quiser, podemos abrir de tal forma que só é permitido adicionar novas linhas, mantendo as linhas originais intactas. Para isso, usamos o modo `a` de `append`:

In [ ]:
meu_texto = open('Dados/meu_texto.txt', 'a')
meu_texto.writelines(['4\n', '5\n']) 
meu_texto.close()

Vamos abrir o arquivo para leitura e verificar se foi excrito corretamente:

In [ ]:
meu_texto = open('Dados/meu_texto.txt')
print(meu_texto.read())
meu_texto.close()

### Abrindo Arquivos com `with`

Ao abrir arquivos, é possível que esqueçamos de fechar após utiliá-lo. Para evitar isso podemos usar o gerenciador de contexto (*context manager*) `with`, que fecha o arquivo após o término da execução do bloco de código:

In [ ]:
with open('Dados/meu_texto.txt') as f:
    print(f.read())
    
print('O arquivo foi fechado?', f.closed)

> Quais são as palavras mais exóticas, ou seja, as menos usadas no Dom Casmurro? E as mais usadas? 
>
> Crie um contador de palavras pare responder as perguntas acima. Aproveite também para escrever o resultado em um arquivo no disco. 

## Obtendo Arquivos da Internet 

Em algumas ocasiões, os dados que vamos precisar estará disponibilizado na Internet e precisaremos trazê-los para o disco. Como exemplo, vamos fazer o *download*  dos dados dos passsageiros do Titanic.

In [ ]:
from urllib.request import urlretrieve

url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv'

urlretrieve(url, 'Dados/titanic.csv')

E agora verificamos se os dados foram trazidos com sucesso:

In [ ]:
with open('Dados/titanic.csv') as f:
    for i in range(6):
        print(f.readline()) # as 6 primeiras linhas

O arquivo foi trazido com sucesso e foi possível abrir e mostra as 6 primeiras linhas. 

## Arquivos Tabulares (CSV, ...)

O arquivo que baixamos no exemplo acima é do tipo CSV, *comma separated values* ou, no bom português, valores separados por vírgula. A terminação `.csv` é só uma convenção para facilitar a identificação do arquivo. 

O Python possui um módulo chamado `csv` que nos ajuda a interagir com este tipo de arquivo:

In [ ]:
import csv
with open('Dados/titanic.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    titanic = [row for row in reader]
    
print(titanic[:10])

Para escrever um arquivo CSV, usamos o `csv.writer`:

In [ ]:
with open('Dados/exemplo.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t') # Usamos uma tabulação como separador
    for i in range(11):
        writer.writerow([i**j for j in range(1, 11)])

In [ ]:
with open('Dados/exemplo.csv') as f:
    print(f.read())

Quando abrimos o arquivo com dados do Titanic, os dados foram salvos na memória como uma lista de listas. Também podemos salvar como uma lista de dicionários usando o método `Dictreader`: 

In [ ]:
with open('Dados/titanic.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    titanic = [row for row in reader]

print(titanic[:5])

Se você notar bem, os dados retornados não estão em um dicionário, como falei antes, mas em um objeto do tipo `OrderedDict`. Este 'um tipo especial de dicionário disponível no módulo `collections`. A diferença do `OrderedDict` para o dicionário básico é que ele mantém a ordem dos objetos inseridos, como podemos ver abaixo:

In [ ]:
print('--Primeira linha--')
print(titanic[0])
print('--Segunda linha--')
print(titanic[1])

O método `DictReader`, por padrão, usa a primeira linha do arquivo CSV para criar as chaves dos dicionários. Assim, podemos descobrir o atributo `age` do passageiro registrado na primeira linha do arquivo CSV da seguinte forma:

In [ ]:
print(titanic[0]['age'])

E, para passar de um dicionário para um arquivo CSV, podemos usar o  `DictWriter`:

In [ ]:
with open('Dados/exemplo.csv', 'w') as csvfile:
    cabecalho = ['i', 'i^2', 'i^3']
    writer = csv.DictWriter(csvfile, fieldnames=cabecalho)
    writer.writeheader() # escreve o cabecalho
    for i in range(11):
        writer.writerow({'i': i, 'i^2': i**2, 'i^3': i**3})

Vamos verificar se foi escrito corretamente:

In [ ]:
with open('Dados/exemplo.csv') as f:
    print(f.read())

Como não deifinimos o delimitador, o `DictWriter` usou a vírgula como padrão.

## Arquivos JSON

De acordo com o [site oficial][json-website], o formato JSON é definido como:

> JSON (JavaScript Object Notation - Notação de Objetos JavaScript) é uma formatação leve de troca de dados. Para seres humanos, é fácil de ler e escrever. Para máquinas, é fácil de interpretar e gerar. Está baseado em um subconjunto da linguagem de programação JavaScript, Standard ECMA-262 3a Edição -Dezembro - 1999. JSON é em formato texto e completamente independente de linguagem, pois usa convenções que são familiares às linguagens C e familiares, incluindo C++, C#, Java, JavaScript, Perl, Python e muitas outras. Estas propriedades fazem com que JSON seja um formato ideal de troca de dados.
> 
> JSON está constituído em duas estruturas:
> 
>* Uma coleção de pares nome/valor. Em várias linguagens, isto é caracterizado como um object, record, struct, dicionário, hash table, keyed list, ou arrays associativas.
>* Uma lista ordenada de valores. Na maioria das linguagens, isto é caracterizado como uma array, vetor, lista ou sequência.
>
>Estas são estruturas de dados universais. Virtualmente todas as linguagens de programação modernas as suportam, de uma forma ou de outra. É aceitavel que um formato de troca de dados que seja independente de linguagem de programação se baseie nestas estruturas.

No Python, temos a biblioteca `json` que nos permite traduzir os dados aramzenados em memória no Python para o formato JSON e vice versa:

[json-website]: https://www.json.org/json-pt.html

In [ ]:
import json

dados_exemplo = [
    {'nome':'Gustavo', 'sobrenome':'Bragança', 'idade':34},
    {'nome':'Maria', 'sobrenome':'Silva', 'idade':40},
]

dados_json = json.dumps(dados_exemplo)
print(dados_json)
print(type(dados_json))

O método `dumps` tranformou a lista de dicionários em uma string no formato JSON. 

E para salvar em um arquivo, usamos o método `dump` (sem o `s`) em um arquivo aberto para escrita.

In [ ]:
with open('Dados/exemplo.json', 'w') as f:
    json.dump(dados_exemplo, f)

E podemos verificar que o dicionário foi salvo corretamente.

In [ ]:
with open('Dados/exemplo.json') as f:
    print(f.read())

## Arquivos XML 

O formato XML foi criado pelo *Wolrd Wide Web Consortium* a fim de ter uma linguagem de marcação (*markup language*) que fosse intelígivel para máquinas e seres humanos. Vamos fazer o *download* de um exemplo e abri-lo para vermos sua estrutura:

In [ ]:
urlretrieve('https://www.w3schools.com/xml/simple.xml', 'Dados/simple.xml')
with open('Dados/simple.xml') as f:
    print(f.read())

O arquivo consiste de um cardápio de café da manhã com 5 itens. Cada item está cercado pelas tags `<food>` e `</food>`. Para cada item, há quatro subitens: `name`, `price`, `description` e `calories`, também cercados pelas tags `<...>` e `</...>`.

Vamos usar o Beatiful Soup que é uma biblioteca de terceiros para obter dados de arquivos XML e HTML:

In [ ]:
from bs4 import BeautifulSoup

with open('Dados/simple.xml') as f:
    soup = BeautifulSoup(f.read(), 'lxml')

E vamos verificar a "sopa":

In [ ]:
print(soup.prettify())

Nosso arquivo parece OK. 

In [ ]:
for food in soup.find_all('food'):
    for attr in food.find_all():
        print(f'{attr.name}: {attr.string}')
    print()

Podemos iterar diretamente em cada subitem:

In [ ]:
for name in soup.find_all('name'):
    print(name.string)

Ou também podemos usar o comando `find` e `findall` para encontrar os elementos:

Agora, vamos montar um arquivo XML a partir do nosso dicionário de exemplo:

In [ ]:
soup = BeautifulSoup(features='lxml')

for registro in dados_exemplo:
    nome = soup.new_tag('pessoa', nome=registro['nome'])
    
    sobrenome = soup.new_tag('sobrenome')
    sobrenome.string = registro['sobrenome']
    nome.append(sobrenome)
    
    idade = soup.new_tag('idade')
    idade.string = str(registro['idade'])
    nome.append(idade)    
    
    soup.append(nome)

print(soup.prettify())

with open('Dados/dado_exemplo.xml', 'w') as f:
    f.write(str(soup))

Veja que aqui eu coloquei o `nome` dentro do item. Vamos ver como ficou:

In [ ]:
with open('Dados/dado_exemplo.xml') as f:
    print(f.read())

> Como faríamos para estruturar os dados dos passageiros do Titanic em XML?

## Arquivos HTML

Em alguns casos, vamos encontrar informações úteis em páginas da web servidas em HTML. E, assim, para obter estes dados, podemos construir um raspador de web (*web scraping*) com o Beautiful Soup.

Vamos ver o exemplo da [documentação.](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [ ]:
# HTML de exemplo
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

E se renderizassemos nosso HTML em um navegados, ele se pareceria com o resultado abaixo.

In [ ]:
from IPython.core.display import HTML, display

display(HTML(html_doc))

Assim como no XML, o primeiro passo é criar a *sopa*:

In [ ]:
soup = BeautifulSoup(html_doc, 'html.parser')

Note que o segundo atributo mudou e se refere ao tipo de arquivo que estamos tratando.

Podemos acessar diversos atributos desta página, começando pelo título:

In [ ]:
print(soup.title)
print(soup.title.name)
print(soup.title.string)
print(soup.title.parent.name)

Podemos acessar o parágrafo `<p>` e seus elements:

In [ ]:
print(soup.p)
print(soup.p['class'][0])
print(soup.p.text)

Podemos achar por elemento:

In [ ]:
soup.find(id='link3')

E também obter todos os links, tarefa essencial quando estamos construindo um `crawler`:

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

E também obter todo o texto:

In [ ]:
print(soup.get_text())

## Bancos de Dados Relacional 

Quando falamos em dados, logo pensamos em um banco de dados relacional. Vamos ver agora como executar *queries* em um banco e, para isso, vamos usar um banco de dados SQLite de exemplo chamado Chinook disponibilizado [aqui](https://www.sqlitetutorial.net/sqlite-sample-database/).

In [ ]:
import zipfile

filename, response = urlretrieve('https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip', 'Dados/chinook.zip')

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('Dados/')

Vamos usar um banco SQLite, mas outros bancos relacionais seguem a mesma rotina:
1. Criamos uma conexão,
1. Criamos um cursor,
1. Com o cursor, executamos a query,
    * Se for um `SELECT`, recuperamos o resultado.
1. Após terminar, fechamos o cursor e a conexão.

In [ ]:
import sqlite3

conn = sqlite3.connect('Dados/chinook.db')

cursor = conn.cursor()

cursor.execute(
    """
    SELECT * FROM tracks limit 10
    """
)
print(cursor.fetchall())

cursor.close()
conn.close()

Podemos iniciar uma nova base também:

In [ ]:
from datetime import datetime
import random

conn = sqlite3.connect('Dados/meu_banco.db')

cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS acoes
             (created_at timestamp, trans text, simbolo text, qtd real, preco real)''')
cursor.execute(
    "INSERT INTO acoes (created_at, trans, simbolo, qtd, preco) VALUES (?, ?, ?, ?, ?)", 
    (
        datetime.now(), 
        random.choice(['COMPRA', 'VENDE']),
        'GOOG',
        random.randint(1, 10)*100, 
        random.uniform(1400, 1500))
)
conn.commit()

cursor.execute("Select * from acoes")
print(cursor.fetchall())

cursor.close()
conn.close()

## APIs REST

Hoje em dia, muitos aplicativos WEB se comunicam entre si usando através de APIs (*Acces Point Interface*) usando o protocolo REST (*Representational State Transfer*) que é uma convenção em torno de 4 *verbos* que são usados para:

* `GET`: obter um recurso.
* `POST`: cria um recurso  .
* `PUT`: atualiza um recurso.
* `DELETE`: deleta um recurso.


Vamos ver o exemplo de dois, `GET` e `POST`:

In [ ]:
import requests

r = requests.get('https://jsonplaceholder.typicode.com/posts?id=1')
print(r.status_code)
assert r.status_code == 200 # success

Acima fizemos um `GET` a fim de obter um recurso, que neste caso foi um `post` cujo `id` é igual a 1. Tudo que está além do ponto de interrogação é chamado de *query string* e é utilizado para filtrar o resultado. 

Obtemos a resposta em JSON no atributo `.text` da resposta.

In [ ]:
print(r.text)

Agora vamos fazer um `POST` e adicionar informação na API. A estrutura para envio é um pouco mais complexa: além do *endpoint* (endereço da API), precisamos do dado a ser enviado em JSON e informar no cabeçalho o formato do conetúdo sendo enviado.

In [ ]:
r = requests.post(
    'https://jsonplaceholder.typicode.com/posts',
    data =  json.dumps(
        dict(
            title= 'foo',
            body= 'bar',
            userId= 1
        )
    ),
        
    headers= {
      "Content-type": "application/json; charset=UTF-8"
    }
)
assert r.status_code == 201 # created